In [1]:
import geopandas as gpd
import pandas as pd
import requests

https://figshare.com/articles/dataset/Data_for_Methodological_Foundation_of_a_Numerical_Taxonomy_of_Urban_Form_research_paper/16897102

In [2]:
url = "https://figshare.com/ndownloader/files/31252828"

In [ ]:
r = requests.get(url)

In [ ]:
with open("geoms.gpkg.zip", "wb") as f:
    f.write(r.content)

Unzip

In [2]:
buildings = gpd.read_file("prg_geometry.gpkg", engine='pyogrio', layer="buildings")

In [3]:
buildings = buildings.set_geometry(buildings.centroid)

In [4]:
clusters = pd.read_csv("https://figshare.com/ndownloader/files/31250203", index_col=0)

In [5]:
buildings = buildings.merge(clusters, on="uID")

In [6]:
buildings.cluster.nunique()

10

In [7]:
mapping = {
    0: "large-scale industry",
    1: "single family housing",
    2: "fringe development",
    3: "grids",
    4: "housing estates",
    5: "periphery",
    6: "grids",
    7: "medieval city",
    8: "single family housing",
    9: "small-scale row houses",
}

In [8]:
buildings["cluster"] = buildings["cluster"].map(mapping)

In [9]:
prague = gpd.read_file(
    "https://martinfleischmann.net/sds/chapter_04/data/zsj_prague_2021.gpkg"
)

In [10]:
buildings = buildings.sjoin(prague.to_crs(buildings.crs).drop(columns="n_people"), how='left')

In [13]:
r = requests.get("https://figshare.com/ndownloader/files/31250227")
with open("prg_primary.csv.gz", "wb") as f:
    f.write(r.content)

In [11]:
data = pd.read_csv("prg_primary.csv.gz", index_col=0)

In [12]:
data = buildings.merge(data, on="uID")

In [13]:
data

,uID,geometry,cluster,index_right,NAZ_ZSJ,NAZ_KU,bID,stcOri,sdcLAL,sdcAre,...,ldsMSL,ldsRea,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB
0,0,POINT (-749841.681 -1052279.951),large-scale industry,760.0,U cementárny,Radotín,0,7.603973,33.817996,342.233358,...,500.099794,1.006508e+06,397667.372035,3615.243681,0.404219,0.724434,880.163669,28.214414,0.000553,0.000108
1,1,POINT (-744432.289 -1042699.409),medieval city,34.0,Horní malostranský obvod,Malá Strana,1,3.712491,58.703914,1435.882458,...,129.395330,3.504396e+05,262570.110986,2902.374890,0.392367,0.742951,714.928910,8.174868,0.003790,0.000369
2,2,POINT (-733300.261 -1048136.856),periphery,553.0,Dolní Měcholupy-střed,Dolní Měcholupy,4,12.121107,34.814040,433.237769,...,245.801133,4.159394e+05,56345.646398,1310.808733,0.264617,0.799244,389.760230,28.532001,0.004577,0.000461
3,3,POINT (-742468.177 -1039691.997),periphery,334.0,Trojský obvod,Troja,5,7.700002,45.376335,839.521945,...,334.823124,1.586278e+06,93980.927007,1236.106936,0.694690,0.994124,119.993680,6.159727,0.004045,0.000638
4,4,POINT (-740093.985 -1043857.813),grids,62.0,Vrch Svatého kříže,Žižkov,6,29.080840,135.981870,7058.965193,...,133.672812,3.444156e+05,7058.965193,328.785377,0.486059,1.039719,31.062697,29.080840,0.006083,0.000142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140310,140457,POINT (-742330.100 -1043654.354),medieval city,30.0,Jindřišský obvod,Nové Město,5488,28.178572,98.965803,3505.778714,...,117.971921,3.082993e+05,56606.217751,951.571799,0.596143,1.009986,94.926096,35.086045,0.008407,0.000654
140311,140458,POINT (-745114.729 -1036851.623),periphery,313.0,Suchdol-střed,Suchdol,1139,24.776814,92.897524,5114.085960,...,129.984846,4.016710e+05,40933.111154,1380.026247,0.224597,0.664578,447.681498,10.487112,0.005797,0.000220
140312,140459,POINT (-741484.617 -1045909.408),grids,143.0,Nuselský obvod,Nusle,4960,15.112430,34.559509,520.511963,...,93.310638,1.411415e+05,9762.288285,426.012538,0.550692,0.956055,61.986071,4.259050,0.011737,0.002663
140313,140460,POINT (-741509.029 -1045897.128),grids,143.0,Nuselský obvod,Nusle,4960,20.447865,19.148696,193.498945,...,93.310638,1.411415e+05,9762.288285,426.012538,0.550692,0.956055,61.986071,4.259050,0.011737,0.002663


In [17]:
data[["geometry", "cluster", "sicFAR", "sdbHei", "ssbCCo", "stbSAl", "ltbIBD", "ldbPWL", "NAZ_ZSJ", "NAZ_KU"]].rename(columns={
    "sicFAR": "floor_area_ratio",
    "sdbHei": "height",
    "ssbCCo": "compactness",
    "stbSAl": "street_alignment",
    "ltbIBD": "interbuilding_distance",
    "ldbPWL": "block_perimeter_wall_length",
    "NAZ_ZSJ": "basic_settlement_unit",
    "NAZ_KU": "cadastral zone"
}).dropna(subset="basic_settlement_unit").to_file("../../chapter_11/data/prg_building_locations.gpkg", engine="pyogrio")

In [16]:
data

,uID,geometry,cluster,index_right,NAZ_ZSJ,NAZ_KU,bID,stcOri,sdcLAL,sdcAre,...,ldsMSL,ldsRea,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB
0,0,POINT (-749841.681 -1052279.951),large-scale industry,760.0,U cementárny,Radotín,0,7.603973,33.817996,342.233358,...,500.099794,1.006508e+06,397667.372035,3615.243681,0.404219,0.724434,880.163669,28.214414,0.000553,0.000108
1,1,POINT (-744432.289 -1042699.409),medieval city,34.0,Horní malostranský obvod,Malá Strana,1,3.712491,58.703914,1435.882458,...,129.395330,3.504396e+05,262570.110986,2902.374890,0.392367,0.742951,714.928910,8.174868,0.003790,0.000369
2,2,POINT (-733300.261 -1048136.856),periphery,553.0,Dolní Měcholupy-střed,Dolní Měcholupy,4,12.121107,34.814040,433.237769,...,245.801133,4.159394e+05,56345.646398,1310.808733,0.264617,0.799244,389.760230,28.532001,0.004577,0.000461
3,3,POINT (-742468.177 -1039691.997),periphery,334.0,Trojský obvod,Troja,5,7.700002,45.376335,839.521945,...,334.823124,1.586278e+06,93980.927007,1236.106936,0.694690,0.994124,119.993680,6.159727,0.004045,0.000638
4,4,POINT (-740093.985 -1043857.813),grids,62.0,Vrch Svatého kříže,Žižkov,6,29.080840,135.981870,7058.965193,...,133.672812,3.444156e+05,7058.965193,328.785377,0.486059,1.039719,31.062697,29.080840,0.006083,0.000142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140310,140457,POINT (-742330.100 -1043654.354),medieval city,30.0,Jindřišský obvod,Nové Město,5488,28.178572,98.965803,3505.778714,...,117.971921,3.082993e+05,56606.217751,951.571799,0.596143,1.009986,94.926096,35.086045,0.008407,0.000654
140311,140458,POINT (-745114.729 -1036851.623),periphery,313.0,Suchdol-střed,Suchdol,1139,24.776814,92.897524,5114.085960,...,129.984846,4.016710e+05,40933.111154,1380.026247,0.224597,0.664578,447.681498,10.487112,0.005797,0.000220
140312,140459,POINT (-741484.617 -1045909.408),grids,143.0,Nuselský obvod,Nusle,4960,15.112430,34.559509,520.511963,...,93.310638,1.411415e+05,9762.288285,426.012538,0.550692,0.956055,61.986071,4.259050,0.011737,0.002663
140313,140460,POINT (-741509.029 -1045897.128),grids,143.0,Nuselský obvod,Nusle,4960,20.447865,19.148696,193.498945,...,93.310638,1.411415e+05,9762.288285,426.012538,0.550692,0.956055,61.986071,4.259050,0.011737,0.002663
